In [73]:
import csv
import math
import string
import os

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from PIL import Image
from IPython.display import display, Markdown

from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn import svm
from sklearn import preprocessing
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.neighbors import NearestNeighbors as KNN
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import Lasso
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import Perceptron
from sklearn.linear_model import Ridge
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.kernel_ridge import KernelRidge
from sklearn.gaussian_process.kernels import RBF
from sklearn.gaussian_process.kernels import Matern
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.ensemble import VotingRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.svm import LinearSVR
from sklearn.multioutput import MultiOutputRegressor
from sklearn.cross_decomposition import PLSRegression

from joblib import dump, load

from nltk.stem import PorterStemmer
import nltk
from nltk.corpus import stopwords

import gensim

import torch
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import torch.utils.data as Data
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM
from torch.autograd import Variable

import utils.BoW_data as dp

# Load train/test data

In [12]:
ddir = 'data/'

mlb_tags_train, mlb_tags_test, df = dp.load_dataframe()

df.head()

,image_file,resnet_vector,descriptions,tags,tags_values,tags_vec,word_list,word_vector
0,images_train/5373.jpg,"[-0.8994496464729309, -0.9304700493812561, -2....","[a red train is docked at the station, Several...","[vehicle:train, person:person, indoor:clock, a...","[train, person, clock, handbag]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[red, train, dock, station, sever, peopl, stan...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,images_train/984.jpg,"[-1.3469539880752563, -3.1194605827331543, -0....",[A man with blue jersey holding a baseball bat...,"[person:person, sports:baseball bat]","[person, baseball bat]","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[A, man, blue, jersey, hold, basebal, bat, clo...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,images_train/7127.jpg,"[-3.44549822807312, -1.5245732069015503, -1.00...",[A kitchen decorated in red and white with acc...,"[appliance:refrigerator, appliance:oven, appli...","[refriger, oven, sink, cup, cake, vase]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[A, kitchen, decor, red, white, accessori, A, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,images_train/9609.jpg,"[1.1146496534347534, -2.1671018600463867, 0.09...",[A black and white dog chasing sheep in a fiel...,"[animal:dog, animal:sheep]","[dog, sheep]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[A, black, white, dog, chase, sheep, field, sm...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,images_train/5293.jpg,"[1.6026496887207031, -1.5058174133300781, 3.02...",[Two bears with their mouths open in the water...,[animal:bear],[bear],"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[two, bear, mouth, open, water, A, coupl, bear...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [13]:
mlb_tags_train2, mlb_tags_test2, df_test = dp.load_dataframe(train_or_test='test')

df_test.head()

,image_file,resnet_vector,descriptions,tags,tags_values,tags_vec,word_list,word_vector
0,images_test/152.jpg,"[-0.148756742477417, -0.4813389182090759, -0.6...","[A sign sitting above a store front entrance.,...",[person:person],[person],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[A, sign, sit, store, front, entranc, small, a...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,images_test/901.jpg,"[-2.7743351459503174, -2.609677791595459, -2.2...",[A steam locamotive passes by some houses on a...,[indoor:clock],[clock],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[A, steam, locamot, pass, hous, track, A, smal...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,images_test/1609.jpg,"[-4.7030134201049805, -3.1210086345672607, -0....",[a woman with purple hair is taking a picture ...,"[animal:dog, animal:horse, person:person]","[dog, hors, person]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[woman, purpl, hair, take, pictur, A, hipster,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,images_test/501.jpg,"[-4.7203192710876465, -3.283935546875, -3.4253...",[A GREEN AND BROWN SUITCASE LYING UP ON A LEDG...,[furniture:toilet],[toilet],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[A, green, and, brown, suitcas, ly, UP, ON, A,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,images_test/517.jpg,"[-2.5421407222747803, 0.7139833569526672, -4.4...",[A desk with a small white computer set up on ...,"[animal:cat, kitchen:bowl, furniture:bed]","[cat, bowl, bed]","[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, ...","[A, desk, small, white, comput, set, A, desk, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [14]:
# Have to do this because load_dataframe() is messing up resnet_vectors somehow
with open('data/features_train/features_resnet1000_train.csv', 'r') as fp:
    features_train = [x.strip().split(',') for x in fp.readlines()]
    
features_train = {x[0]: np.array([np.float(n) for n in x[1:]]) for x in features_train}

with open('data/features_test/features_resnet1000_test.csv', 'r') as fp:
    features_test = [x.strip().split(',') for x in fp.readlines()]
    
features_test = {x[0]: np.array([np.float64(n) for n in x[1:]]) for x in features_test}

In [15]:
# Have to do this because load_dataframe() is messing up resnet_vectors somehow
with open('data/features_train/features_resnet1000intermediate_train.csv', 'r') as fp:
    int_features_train = [x.strip().split(',') for x in fp.readlines()]
    
int_features_train = {x[0]: np.array([np.float(n) for n in x[1:]]) for x in int_features_train}

# Have to do this because load_dataframe() is messing up resnet_vectors somehow
with open('data/features_test/features_resnet1000intermediate_test.csv', 'r') as fp:
    int_features_test = [x.strip().split(',') for x in fp.readlines()]
    
int_features_test = {x[0]: np.array([np.float64(n) for n in x[1:]]) for x in int_features_test}

# W2V

In [16]:
df['desc_joined'] = df['descriptions'].apply(lambda x: " ".join(x).split(" "))
df_test['desc_joined'] = df_test['descriptions'].apply(lambda x: " ".join(x).split(" "))

In [17]:
w2v_model = gensim.models.KeyedVectors.load_word2vec_format('./models/GoogleNews-vectors-negative300.bin', binary=True)

In [18]:
ps = PorterStemmer()

stop_words = set(stopwords.words('english'))
stop_words.remove('not')

In [19]:
def get_word2vec(df_in, wmodel):
    
    def prep_doc(doc):

        # Stem and lower all the words
        sentence = [
            ps.stem(word).lower() for word in doc
        ]

        # Remove trailing commas and periods
        sentence = [
            "".join([char for char in word if char not in string.punctuation]) for word in sentence
        ]

        # Remove stopwords, remove any blank words 
        sentence = [
            word for word in sentence if word not in stop_words and word not in  [" ", ""]
        ]
        
        return sentence
    
    def try_get_vec(word):
        try:
            word = wmodel.get_vector(word)
            can_do = True
        except:
            word = np.zeros(300)
            can_do = False
        return can_do
    
    def get_vector(word):
        try:
            word = wmodel.get_vector(word)
        except:
            word = np.zeros(300)
        return word
    
    stops = set(stopwords.words('english'))
    vecs = df_in['desc_joined'].apply(
        lambda desc: np.average([
            get_vector(word) for word in prep_doc(desc)
            if try_get_vec(word)
        ], axis=0)
    )
    
    return np.asarray([vec for vec in vecs], dtype=np.float64)

# Training data prep

In [20]:
X = np.asarray([features_train[x] for x in df['image_file']], dtype=np.float64)
y = get_word2vec(df, w2v_model)

y_true = df['image_file'].to_numpy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# SGD Regressor

Multi output is super slow

In [65]:
multi = MultiOutputRegressor(SGDRegressor())

multi.fit(X_train, y_train)

MultiOutputRegressor(estimator=SGDRegressor(alpha=0.0001, average=False,
                                            early_stopping=False, epsilon=0.1,
                                            eta0=0.01, fit_intercept=True,
                                            l1_ratio=0.15,
                                            learning_rate='invscaling',
                                            loss='squared_loss', max_iter=1000,
                                            n_iter_no_change=5, penalty='l2',
                                            power_t=0.25, random_state=None,
                                            shuffle=True, tol=0.001,
                                            validation_fraction=0.1, verbose=0,
                                            warm_start=False),
                     n_jobs=None)

In [66]:
multi.score(X_test, y_test)

-2.5884428211637805e+23

In [58]:
multi = MultiOutputRegressor(SVR(kernel='poly', C=100, gamma='auto', degree=3, epsilon=.1,
               coef0=1))

multi.fit(X_train, y_train)

MultiOutputRegressor(estimator=SVR(C=100, cache_size=200, coef0=1, degree=3,
                                   epsilon=0.1, gamma='auto', kernel='poly',
                                   max_iter=-1, shrinking=True, tol=0.001,
                                   verbose=False),
                     n_jobs=None)

In [59]:
multi.score(X_test, y_test)

0.31295815966301227

In [ ]:
multi = MultiOutputRegressor(AdaBoostRegressor(random_state=0, n_estimators=100))

multi.fit(X_train, y_train)

In [ ]:
multi.score(X_test, y_test)

Multi SVR - 0.312

# Kernel Ridge

In [21]:
clf = KernelRidge(degree=4)
clf.fit(X_train, y_train)

KernelRidge(alpha=1, coef0=1, degree=4, gamma=None, kernel='linear',
            kernel_params=None)

In [22]:
clf.score(X_test, y_test)

/usr/local/lib/python3.7/site-packages/sklearn/base.py:420: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)


0.40801693389570415

0.408 KernelRidge(degree=4)

# Kernel Ridge RBF

In [37]:
clf = KernelRidge(kernel=RBF(40), alpha=1, degree=4)
clf.fit(X_train, y_train)

KernelRidge(alpha=1, coef0=1, degree=4, gamma=None, kernel=RBF(length_scale=1),
            kernel_params=None)

In [38]:
clf.score(X_test, y_test)

/usr/local/lib/python3.7/site-packages/sklearn/base.py:420: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)


-0.8970433109869271

In [67]:
clf_lin = KernelRidge(alpha=1, degree=4, kernel='rbf')
clf_lin.fit(X_train, y_train)

KernelRidge(alpha=1, coef0=1, degree=4, gamma=None, kernel='rbf',
            kernel_params=None)

In [68]:
clf_lin.score(X_test, y_test)

/usr/local/lib/python3.7/site-packages/sklearn/base.py:420: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)


0.30244644483222466

# PLSR

In [74]:
pls = PLSRegression()
pls.fit(X_train, y_train)

PLSRegression(copy=True, max_iter=500, n_components=2, scale=True, tol=1e-06)

In [75]:
pls.score(X_test, y_test)

/usr/local/lib/python3.7/site-packages/sklearn/base.py:420: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)


0.10463092507910798

### Hyperparam tuning

In [ ]:
parameters = {
    'kernel':('linear', RBF(0.5), RBF(1), RBF(2), Matern(1)),
    'alpha':[0.1, 1, 10],
    'degree': [3,4,5]
}

hyper = GridSearchCV(KernelRidge(), parameters, n_jobs=-1)

In [ ]:
hyper.fit(X_train, y_train)

In [ ]:
print(hyper.best_params_)

y_true, y_pred = y_test, hyper.predict(X_test)
classification_report(y_true, y_pred)

In [ ]:
hyper.score(X_test, y_test)

# Ensemble methods

In [76]:
r1 = KernelRidge(alpha=1, degree=4)
r2 = PLSRegression()

er = VotingRegressor([('lr', r1), ('rf', r2)])
mer = MultiOutputRegressor(er)
mer.fit(X_train, y_train)

VotingRegressor(estimators=[('lr',
                             KernelRidge(alpha=1, coef0=1, degree=4, gamma=None,
                                         kernel='linear', kernel_params=None)),
                            ('rf',
                             PLSRegression(copy=True, max_iter=500,
                                           n_components=2, scale=True,
                                           tol=1e-06))],
                n_jobs=None, weights=None)

In [77]:
er.score(X_test, y_test)

ValueError: Found input variables with inconsistent numbers of samples: [2000, 300]

In [80]:
X_test.shape

(2000, 1000)

In [78]:
regr = BaggingRegressor(base_estimator=KernelRidge(alpha=1, degree=4),
                         n_estimators=10, random_state=0).fit(X, y)

regr.fit(X_train, y_train)

BaggingRegressor(base_estimator=KernelRidge(alpha=1, coef0=1, degree=4,
                                            gamma=None, kernel='linear',
                                            kernel_params=None),
                 bootstrap=True, bootstrap_features=False, max_features=1.0,
                 max_samples=1.0, n_estimators=10, n_jobs=None, oob_score=False,
                 random_state=0, verbose=0, warm_start=False)

In [79]:
regr.score(X_test, y_test)

/usr/local/lib/python3.7/site-packages/sklearn/base.py:420: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)


0.4033278594844159

In [ ]:
r1 = KernelRidge(alpha=1, degree=4)
r2 = MultiOutputRegressor(SVR(kernel='poly', C=100, gamma='auto', degree=3, epsilon=.1,
               coef0=1))

er = VotingRegressor([('lr', r1), ('rf', r2)])
er.fit(X_train, y_train)

In [ ]:
er.score(X_test, y_test)

# NN

In [144]:
solvers = ['lbfgs', 'sgd', 'adam']

nn = MLPRegressor(
    hidden_layer_sizes=(1024, 512),
    max_iter=300,
    solver='adam',
    activation='relu',
    verbose=True
)

In [145]:
nn.fit(X_train, y_train)

Iteration 1, loss = 0.04762700
Iteration 2, loss = 0.00354088
Iteration 3, loss = 0.00291076
Iteration 4, loss = 0.00250705
Iteration 5, loss = 0.00226912
Iteration 6, loss = 0.00213586
Iteration 7, loss = 0.00206379
Iteration 8, loss = 0.00202600
Iteration 9, loss = 0.00200665
Iteration 10, loss = 0.00199700
Iteration 11, loss = 0.00199231
Iteration 12, loss = 0.00199009
Iteration 13, loss = 0.00198903
Iteration 14, loss = 0.00198858
Iteration 15, loss = 0.00198837
Iteration 16, loss = 0.00198829
Iteration 17, loss = 0.00198825
Training loss did not improve more than tol=0.000100 for 10 consecutive epochs. Stopping.


MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
             beta_2=0.999, early_stopping=False, epsilon=1e-08,
             hidden_layer_sizes=(1024, 512), learning_rate='constant',
             learning_rate_init=0.001, max_iter=300, momentum=0.9,
             n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
             random_state=None, shuffle=True, solver='adam', tol=0.0001,
             validation_fraction=0.1, verbose=True, warm_start=False)

In [147]:
nn.score(X_test, y_test)

/usr/local/lib/python3.7/site-packages/sklearn/base.py:420: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)


-0.0013719643188388727

# Model validation

In [81]:
model = regr

In [82]:
def score_prediction(true, pred):

    if true not in pred:
        return 0.0
    
    ind = pred.index(true)
    
    return (20 - ind) / 20


def nearest_neighbors(sample, targets):
    """Returns index positions of nearest neighbors"""
    distances = [np.linalg.norm(t-sample) for t in targets]
    distances_sorted = np.argsort(distances)
    return distances_sorted[:20]


def furthest_neighbors(sample, targets):
    """Returns index positions of nearest neighbors"""
    distances = [np.linalg.norm(t-sample) for t in targets]
    distances_sorted = np.argsort(distances)
    return distances_sorted[-20:]

In [83]:
pred_embeddings = [model.predict(vec.reshape(1, -1)) for vec in X_test]
pred_indices = [nearest_neighbors(sample, pred_embeddings) for sample in y_test]

preds = [
    [y_true[ind] for ind in pred]
    for pred in pred_indices
]

In [84]:
score = sum(
    [score_prediction(y_true[ind], pred) for ind, pred in enumerate(preds)]
           ) / len(preds)

In [85]:
print(score)

0.5686250000000019


# Test predictions

In [ ]:
model = regr

In [ ]:
resout = np.asarray([features_test[x] for x in df_test['image_file']], dtype=np.float64)
decout = get_word2vec(df_test, w2v_model)

In [ ]:
test_pred_embeddings = [model.predict(vec.reshape(1, -1)) for vec in resout]
test_pred_indices = [nearest_neighbors(sample, test_pred_embeddings) for sample in decout]

y_targets = df_test['image_file'].to_numpy()

preds = [
    [y_targets[ind] for ind in pred]
    for pred in test_pred_indices
]

preds = [[file.split('/')[-1] for file in pred] for pred in preds]

In [ ]:
desc_txt_files = df_test['image_file'].apply(
        lambda x: x.split('/')[-1].replace('jpg', 'txt')
    )

In [ ]:
out_preds = list(zip(desc_txt_files.to_numpy(), preds))

In [ ]:
with open('regr.csv', 'w+') as f:
    f.write('Descritpion_ID,Top_20_Image_IDs\n')
    for pred in out_preds:
        f.write(f"{pred[0]},{' '.join(pred[1])}\n")